In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator
from scipy.stats import *

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('./train_2.csv').fillna(0)

In [5]:
key = pd.read_csv('./key_trunc.csv')

In [6]:
sub = pd.read_csv("./sample_submission_2.csv")

In [59]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [7]:
def get_median(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider - 1:])

In [8]:
def get_mean(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider:]) 
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider - 1:])

In [9]:
def get_std(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider - 1:])

In [10]:
def get_min(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider - 1:])        

In [11]:
def get_max(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider - 1:])        

In [12]:
def get_mean_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(len(row) - rows_to_consider)

In [13]:
def get_median_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].median()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].median()).repeat(len(row) - rows_to_consider)

In [35]:
# def get_mean_diff(row, last_n_days):
#     return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [36]:
# def get_last_day_visits(row):
#     return row[1:]

In [14]:
def get_long_stats(func_type, df, df_melt, rows_to_consider, is_it_train = None, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.apply(get_median, last_n_days = last_n_days, rows_to_consider = rows_to_consider,is_it_train = is_it_train, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.apply(get_mean, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
    
    elif func_type == "std":
        rolling_stats = df.apply(get_std, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "min":
        rolling_stats = df.apply(get_min, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "max":
        rolling_stats = df.apply(get_max, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "mean_overall":
        rolling_stats = df.apply(get_mean_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "median_overall":
        rolling_stats = df.apply(get_median_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
        
    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[rows_to_consider:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [25]:
def generate_train_melt(train):
    
    train_melt = pd.melt(pd.concat([train.Page, 
                                    train.iloc[:, -180:]],
                                    axis = 1),
                                    id_vars=['Page'], 
                                    var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()
    
    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("mean_overall", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    train_melt = get_long_stats("median_overall", train, train_melt, rows_to_consider=-180, is_it_train = 1, last_n_days=60)
    print train_melt.head()
        
    train_melt["date_num"] = train_melt.apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.apply(get_weekday, axis = 1)
    print train_melt.head()
    
    return train_melt 

def get_lang_features(train_melt):
    
    train_melt = pd.merge(train_melt, pd.concat([train.Page, train.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    train_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = train_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.median))

    lang_feats.columns = ["language", "date", "lang_mean_30_median", "lang_mean_60_median", "lang_median_30_median", "lang_median_60_median", "lang_min_30_median", "lang_min_60_median", "lang_max_30_median", "lang_max_60_median"]
        
    train_melt = pd.merge(train_melt, lang_feats, on = ["language", "date"])
    
    return train_melt

def get_weekday_features(train_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    train_melt = pd.merge(train_melt, weekday_feats, on = ["weekday", "Page"])
    
    return train_melt
    
# def get_lang_mean(train_melt):
    
#     language_page_melt = train_melt[["language", "date", "visits"]]
    
#     language_feats = pd.DataFrame(language_page_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

#     language_feats.columns = ["language", "date", "lang_mean_visits"]
        
#     train_melt = pd.merge(train_melt, language_feats, on = ["language", "date"])
    
#     return train_melt

train_melt = generate_train_melt(train)
# train_melt = get_lang_features(train_melt)
train_melt = get_weekday_features(train_melt)
train_melt.to_csv("train_feats.csv", index = False)

                                                Page        date  visits  \
0            2NE1_zh.wikipedia.org_all-access_spider  2017-03-05    16.0   
1             2PM_zh.wikipedia.org_all-access_spider  2017-03-05    15.0   
2              3C_zh.wikipedia.org_all-access_spider  2017-03-05     5.0   
3         4minute_zh.wikipedia.org_all-access_spider  2017-03-05    23.0   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...  2017-03-05    12.0   

     mean_30    mean_60  
0  19.666667  25.533333  
1  30.233333  27.900000  
2   3.500000   3.766667  
3  22.966667  17.450000  
4  42.633333  45.366667  
                                                Page        date  visits  \
0            2NE1_zh.wikipedia.org_all-access_spider  2017-03-05    16.0   
1             2PM_zh.wikipedia.org_all-access_spider  2017-03-05    15.0   
2              3C_zh.wikipedia.org_all-access_spider  2017-03-05     5.0   
3         4minute_zh.wikipedia.org_all-access_spider  2017-03-05    23.0   
4  52_H

In [16]:
def get_source(row):
    return row.Page.splitgit["."][-1].split("_")[-1]

In [17]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [18]:
def get_month(row):
    return row.date.split("-")[1]

In [19]:
def get_date(row):
    return row.date.split("-")[2]

In [20]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [44]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [21]:
def get_lang_features_test(test_melt):
    
    test_melt = pd.merge(test_melt, pd.concat([overall_df.Page, overall_df.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    test_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = test_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.median))
    
    lang_feats.columns = ["language", "date", "lang_mean_30_median", "lang_mean_60_median", "lang_median_30_median", "lang_median_60_median", "lang_min_30_median", "lang_min_60_median", "lang_max_30_median", "lang_max_60_median"]
        
    test_melt = pd.merge(test_melt, lang_feats, on = ["language", "date"])
    
    return test_melt

In [22]:
def get_weekday_features_test(train_melt, test_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    test_melt = pd.merge(test_melt, weekday_feats, on = ["weekday", "Page"], how="left")
    
    return test_melt

In [47]:
# def get_last_day_visits_test(row, last_index):
    
#     return row.iloc[last_index - 1]

In [23]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [24]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0
    
    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)

    return bst

In [77]:
# label_train_melt = np.log1p(train_melt["visits"])
label_train_melt = train_melt["visits"]

# label_train_melt = boxcox(np.array(label_train_melt), lmbda=0.094) #lambda found from default boxcox on outlier-stripped data

# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
# train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [83]:
train_melt.drop("date_num", axis = 1, inplace=True)
train_melt.drop("weekday", axis = 1, inplace=True)

In [84]:
# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# bst = run_xgb(x_train, label_train, x_valid, label_valid)

bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:63660.4	train-smape:161.974
Multiple eval metrics have been passed: 'train-smape' will be used for early stopping.

Will train until train-smape hasn't improved in 10 rounds.
[10]	train-rmse:48434.9	train-smape:78.3514
[20]	train-rmse:41524.9	train-smape:55.2278
[30]	train-rmse:38689.9	train-smape:49.3566
[40]	train-rmse:37594.9	train-smape:48.9122
Stopping. Best iteration:
[32]	train-rmse:38379.2	train-smape:48.5358



In [85]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

                  feature    fscore
0                 mean_60  0.003197
1                  min_60  0.023181
2                  std_60  0.026379
3       median_overall_60  0.040368
4                  max_60  0.044365
5         mean_overall_60  0.045564
6                  std_30  0.053957
7               median_30  0.059153
8                  max_30  0.062750
9                 mean_30  0.079936
10              median_60  0.090727
11                 min_30  0.115907
12  weekday_median_visits  0.127498
13    weekday_mean_visits  0.227018


In [245]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# # x_train = x_train.fillna(0)
# # x_valid = x_valid.fillna(0)
# reg.fit(x_train, label_train)

reg.fit(train_melt, label_train_melt)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [243]:
# prediction = reg.predict(x_valid)
prediction = reg.predict(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

In [244]:
y_true = np.array(label_valid)
y_pred = np.array(prediction)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

105.106730814


In [87]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [88]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = np.nan

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [57]:
def generate_test_melt(overall_df):
        
    pred_index = overall_df.columns.tolist().index('2017-09-13') 
    
    test_melt = pd.melt(pd.concat([overall_df.Page, overall_df.iloc[:, pred_index:]],axis = 1),
                                id_vars=['Page'], 
                                var_name="date", value_name="visits")
    
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("mean_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    test_melt = get_long_stats("median_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    
    test_melt["date_num"] = test_melt.apply(get_date, axis = 1)
    test_melt["weekday"] = test_melt.apply(get_weekday, axis = 1)

    return test_melt 

In [58]:
# overall_df = get_overall_df()

test_melt = generate_test_melt(overall_df)

# test_melt = get_lang_features_test(test_melt)

test_melt = get_weekday_features_test(train_melt, test_melt)

test_melt.drop("visits", axis = 1, inplace=True)

KeyError: "['Page' 'visits'] not in index"

In [67]:
last_rows_df = pd.DataFrame(np.unique(test_melt.Page))
last_rows_df.columns = ["Page"]

for i in test_melt.columns.tolist():

    if i not in ["Page", "date", "language", "date_num", "weekday"]:
        
        print i
        test_melt['date'] = pd.to_datetime(test_melt['date'])

        sub_test_melt = test_melt[["Page", "date", i]]

        non_na_sub = sub_test_melt[np.isfinite(sub_test_melt[i])]
        last_date = non_na_sub.loc[non_na_sub['date'].idxmax()]["date"]

        last_rows = non_na_sub[non_na_sub["date"] == last_date][["Page", i]]
        last_rows_df = pd.merge(last_rows_df, last_rows, on="Page")

mean_30
mean_60
median_30
median_60
std_30
std_60
min_30
min_60
max_30
max_60
mean_overall_60
median_overall_60
weekday_mean_visits
weekday_median_visits


In [74]:
test_melt = pd.merge(test_melt, last_rows_df, on = "Page")

for i in train_melt.columns.tolist():

    if i not in ["Page", "date", "language", "date_num", "weekday", "visits"]:

        test_melt[i + "_x"].fillna(test_melt[i + "_y"], inplace = True)
        test_melt[i] = test_melt[i + "_x"]

        test_melt.drop(i + "_x", axis = 1, inplace=True)
        test_melt.drop(i + "_y", axis = 1, inplace=True)    

In [76]:
test_melt.to_csv("test_feats.csv", index = False)

In [6]:
test_melt = pd.read_csv("test_feats.csv")

In [89]:
test_melt.date_num = pd.to_numeric(test_melt.date_num)
prediction = get_preds(test_melt[train_melt.columns.tolist()])

# prediction = invboxcox(get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]]), ld = 0.094)

In [99]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_melt["Visits"] = np.rint(prediction)
# pred_df = pd.concat([test_melt[["Page", "date"]], prediction], axis = 1)
pred_df = test_melt[["Page", "date", "Visits"]]

pred_df['date'] = pd.to_datetime(pred_df['date'])
key['date'] = pd.to_datetime(key['date'])

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_xgb_stage2_2.csv", index = False)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
from scipy.stats import *
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt if i > 4 and i < 1871])) #Outlier 10% to 90% only
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt]))

bc_label = boxcox(np.array(label_train_melt), lmbda=0.094)

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/morestats.py:516: RuntimeWarning: divide by zero encountered in log
  y = where(lmbda == 0, log(x), (x**lmbda - 1)/lmbda)


In [8]:
def invboxcox(y,ld):
    if ld == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(ld*y+1)/ld))

In [10]:
# np.percentile(label_train_melt, 90)
# np.percentile(label_train_melt, 10)
# len([i for i in label_train_melt if i > 4 and i < 1871])

# import matplotlib.pyplot as plt
# plt.hist(bc_label.tolist())
# plt.show()

# bc_label

# import scipy.special.inv_boxcox
invboxcox(bc_label, ld = 0.094)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([ 18.,  11.,   3., ...,   0.,   0.,   0.])